In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('../input/weather-dataset-rattle-package/weatherAUS.csv')
data['RainTomorrow'] = data['RainTomorrow'].map({'No':0, 'Yes':1})
data['RainToday'] = data['RainToday'].map({'No':0, 'Yes':1})

In [ ]:
columns = ['Date','Location','Temp3pm','Temp9am','Pressure3pm','RISK_MM']
data.drop(columns, axis=1, inplace=True)
data = pd.get_dummies(data)
siz = data.shape[1] - 1 
X = data.iloc[:, :-1].values
y = data.iloc[:, siz].values

In [ ]:
data.head()

In [ ]:
#missing data
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
#Imputation
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
imputer = imputer.fit(X)
X = imputer.transform(X)

In [ ]:
def plot_corr(data, size):
    corr = data.corr()
    fig, ax = plt.subplots(figsize=(size,size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)), corr.columns)
    plt.yticks(range(len(corr.columns)), corr.columns)

plot_corr(data, data.shape[1])
data.corr()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
X_train = std.fit_transform(X_train)
X_test = std.transform(X_test)

In [ ]:
print(y_train.shape)
print(X_train.shape)

In [ ]:
#Fitting model
from sklearn.naive_bayes import GaussianNB   
NB = GaussianNB()  
NB.fit(X_train, y_train)

In [ ]:
#Predicting test result
y_pred_nb = NB.predict(X_test)

#Predicting training result
y_pred_train_nb = NB.predict(X_train)

In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_nb)
true_pred = cm[1][1] + cm[0][0]
neg_pred = cm[1][0] + cm[0][1]
print(cm)
print('True prediction: ', true_pred, ' False prediction: ', neg_pred)
print('Accuracy of true prediction: ', (true_pred/X_test.shape[0])*100, '%')
print('Accuracy of false prediction: ', (neg_pred/X_test.shape[0])*100, '%')

In [ ]:
from sklearn import metrics
print ("Accuracy: ({0:.4f})".format(metrics.accuracy_score(y_train, y_pred_train_nb)))

#Performance on testing data
from sklearn import metrics
print ("Accuracy: ({0:.4f})".format(metrics.accuracy_score(y_test, y_pred_nb)))

In [ ]:
from keras import Sequential
from keras.layers import Dense

classifierNN = Sequential()

#First Hidden Layer
classifierNN.add(Dense(2, activation='relu', kernel_initializer='random_normal', input_dim=62))

#Second Hidden Layer
classifierNN.add(Dense(2, activation='relu', kernel_initializer='random_normal'))

#Output Layer
classifierNN.add(Dense(1, activation='sigmoid',  kernel_initializer='random_normal'))

classifierNN.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

classifierNN.fit(X_train,y_train, batch_size=10, epochs=5)

In [ ]:
eval_model=classifierNN.evaluate(X_train, y_train)
eval_model

In [ ]:
y_pred_NN=classifierNN.predict(X_test)
y_pred_NN =(y_pred_NN>0.5)

In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_NN)
true_pred = cm[1][1] + cm[0][0]
neg_pred = cm[1][0] + cm[0][1]
print(cm)
print('True prediction: ', true_pred, ' False prediction: ', neg_pred)
print('Accuracy of true prediction: ', (true_pred/X_test.shape[0])*100, '%')
print('Accuracy of false prediction: ', (neg_pred/X_test.shape[0])*100, '%')